From modin main and snowpark python repo: `pip install -e .`

In [1]:
import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVJdc9owEPwrHvUZSzaQpBogQ0PTuE3BiaHp8KbYMtEgS6pOxri%2FvjIfnfQhmb5pTru3e7c3ut5XMthxC0KrMYpCggKucl0ItRmj1fK2d4UCcEwVTGrFx6jlgK4nI2CVNHRauxf1yH%2FVHFzgGymg3ccY1VZRzUAAVaziQF1Os%2Bn3exqHhDIAbp2XQydKAcJrvThnKMZN04RNP9R2g2NCCCYfsUd1kA%2FolYR5X8NY7XSu5Zmy9zO9IRFhMugkPMIrpCfiJ6GOK3hP5fkIAnq3XKa9dJEtUTA9T3ejFdQVtxm3O5Hz1eP90QB4B9l88XS3WGWfQ1C6KSXb8lxXpna%2BW%2BhfuOQFlnoj%2FI6S2RiZrSjUti3rS%2FOzvHAPX3Sbi3SVDL%2Blg3Juqr7MyK5lUbxau7Z6yFHw45xo3CWaANQ8UV2OzpdIPOyRQY%2FEy7hPyYAOL8KraLhGwcznKBRzB%2BbZbGfxWfwO9daxgzlmDP7rG%2FP99qopobTt10QNkqd1bC4xgMZdTOh4KfRgwE7%2Bd%2F4Rfs06Hdvc7z%2BZpVqKvA1uta2YezueKIwOFVH0ygOU8ooJOS0KywF8TFLq5sZy5vxNO1tzhCdH1X%2BvevIH&RelayState=ver%3A1-hint%3

# Example 1: Start with small dataset perform some processing, then join with large data

In [2]:
# emphemeral dataframe/lookup table 
# List of U.S. federal holidays
us_holidays = [
    ("New Year's Day", "2025-01-01"),
    ("Martin Luther King Jr. Day", "2025-01-20"),
    ("Presidents' Day", "2025-02-17"),
    ("Memorial Day", "2025-05-26"),
    ("Juneteenth National Independence Day", "2025-06-19"),
    ("Independence Day", "2025-07-04"),
    ("Labor Day", "2025-09-01"),
    ("Columbus Day", "2025-10-13"),
    ("Veterans Day", "2025-11-11"),
    ("Thanksgiving Day", "2025-11-27"),
    ("Christmas Day", "2025-12-25")
]

# Create DataFrame
df_us_holidays = pd.DataFrame(us_holidays, columns=["Holiday", "Date"])

# Convert Date column to datetime
df_us_holidays["Date"] = pd.to_datetime(df_us_holidays["Date"])

In [3]:
assert df_us_holidays.get_backend() == 'Pandas'  # with auto, we should expect this to be local

In [4]:
# Add new columns for transformations
df_us_holidays["Day_of_Week"] = df_us_holidays["Date"].dt.day_name()
df_us_holidays["Month"] = df_us_holidays["Date"].dt.month_name()

In [5]:
df_us_holidays

,Holiday,Date,Day_of_Week,Month
0,New Year's Day,2025-01-01,Wednesday,January
1,Martin Luther King Jr. Day,2025-01-20,Monday,January
2,Presidents' Day,2025-02-17,Monday,February
3,Memorial Day,2025-05-26,Monday,May
4,Juneteenth National Independence Day,2025-06-19,Thursday,June
5,Independence Day,2025-07-04,Friday,July
6,Labor Day,2025-09-01,Monday,September
7,Columbus Day,2025-10-13,Monday,October
8,Veterans Day,2025-11-11,Tuesday,November
9,Thanksgiving Day,2025-11-27,Thursday,November


In [6]:
%%time
#Note that without auto-switching, this took 2.5 min
for index, row in df_us_holidays.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 119 ms, sys: 4.22 ms, total: 123 ms
Wall time: 121 ms


In [7]:
pandas_df = df_us_holidays.move_to("pandas") # remove this once we have auto-switching

In [8]:
%%time
for index, row in pandas_df.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 121 ms, sys: 5.18 ms, total: 126 ms
Wall time: 124 ms


### Another mini example: generate synthetic data

In [9]:
%%time
# Generate 10 million transactions (This is also very slow with Snowpark pandas)
import uuid
import pandas
dates = pandas.date_range(start="2025-01-01", end="2025-12-31", freq="D")
num_transactions = 10000000
data = {
    "Transaction_ID": [str(uuid.uuid4()) for _ in range(num_transactions)],
    "Date": np.random.choice(dates, num_transactions),
    "Revenue": np.random.uniform(10, 1000, num_transactions)
}
df_transactions = pd.DataFrame(data)

CPU times: user 18.5 s, sys: 9.28 s, total: 27.8 s
Wall time: 27.9 s


In [10]:
assert df_transactions.get_backend() == "Pandas"

### 💡 Automatic switching speeds up loops/iterations on small data + inline creation of dataframes

## Example 2: Demonstrate that when data is prefiltered via SQL, the engine choice changes

In [11]:
# Run the following to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
session.sql('''
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);''').collect()
session.sql('''SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));''').collect()
session.sql('''INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
''').collect()

[Row(number of rows inserted=10000000)]

In [12]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

In [13]:
len(df_transactions)

10000000

In [14]:
# df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [15]:
df_transactions.groupby("DATE").sum()["REVENUE"]

DATE
2024-01-01    10932367.0
2024-01-02    11015069.0
2024-01-03    11041079.0
2024-01-04    11013729.0
2024-01-05    11164649.0
                 ...    
2025-03-29    11231364.0
2025-03-30    11163728.0
2025-03-31    10882805.0
2025-04-01    11138998.0
2025-04-02    10921750.0
Name: REVENUE, Length: 458, dtype: float64

In [16]:
assert df_transactions.get_backend() == "Snowflake"

In [17]:
# Filter to records in last 7 days
df_transactions_filter = pd.read_snowflake("SELECT * FROM revenue_transactions WHERE Date >= DATEADD( 'days', -7, current_date ) and Date < current_date")

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
len(df_transactions_filter)

152771

In [19]:
assert df_transactions_filter.get_backend() == "Pandas" # This should work once auto switching is on

In [20]:
df_transactions_filter.groupby("DATE").sum()["REVENUE"]

DATE
2025-03-26    11009242.0
2025-03-27    10968147.0
2025-03-28    10938323.0
2025-03-29    11231364.0
2025-03-30    11163728.0
2025-03-31    10882805.0
2025-04-01    11138998.0
Name: REVENUE, dtype: float64

### 💡 Automatic switching means that pandas work well for both small and large data

# Example 3: 

Forecast using last year's transaction data via a custom apply function

In [21]:
start_date = pd.Timestamp("2025-10-01")
end_date = pd.Timestamp("2025-10-31")

df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]

In [22]:
len(df_transactions_filtered)

654480

In [23]:
df_transactions_filtered_pandas = df_transactions.move_to("pandas")

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
# Forecasting function using df.apply

def forecast_revenue(df, start_date, end_date):
    # Filter data from last year
    df_filtered = df[(df["DATE"] >= start_date - pd.Timedelta(days=365)) & (df["DATE"] < start_date)]
    
    # Append future dates to daily_avg for prediction
    future_dates = pd.date_range(start=start_date, end=end_date, freq="D")
    df_future = pd.DataFrame({"DATE": future_dates})

    # Group by DATE and calculate the mean revenue
    daily_avg = df_filtered.groupby("DATE")["REVENUE"].mean().reset_index()
    daily_avg["DATE"] = daily_avg["DATE"].astype('datetime64[ns]')
    # Merge future dates with predicted revenue, filling missing values
    df_forecast = df_future.merge(daily_avg, on="DATE", how="left")
    
    import numpy as np
    # Fill missing predicted revenue with overall mean from last year
    df_forecast["PREDICTED_REVENUE"] = np.nan
    df_forecast["PREDICTED_REVENUE"].fillna(daily_avg["REVENUE"].mean(), inplace=True)
    df_forecast["PREDICTED_REVENUE"] = df_forecast["PREDICTED_REVENUE"].astype("float")
    return df_forecast

In [25]:
# Example usage
df_forecast = forecast_revenue(df_transactions, start_date, end_date)
df_forecast

The current operation leads to materialization and can be slow if the data is large!


Transferring data from Pandas to Snowflake ...:   0%|          | 0/2 [00:00<?, ?it/s]

,DATE,REVENUE,PREDICTED_REVENUE
0,2025-10-01,NaN,505.095936
1,2025-10-02,NaN,505.095936
2,2025-10-03,NaN,505.095936
3,2025-10-04,NaN,505.095936
4,2025-10-05,NaN,505.095936
5,2025-10-06,NaN,505.095936
6,2025-10-07,NaN,505.095936
7,2025-10-08,NaN,505.095936
8,2025-10-09,NaN,505.095936
9,2025-10-10,NaN,505.095936


In [26]:
# df_forecast["DATE"] = df_forecast["DATE"].dt.strftime('%Y-%m-%d')

In [27]:
def adjust_for_holiday_weekend(row):
    # For national holidays, revenue down 20% since stores are closed. For weekends, revenue is up 20% due to increased activity.
    if row["DATE"].strftime('%Y-%m-%d') in list(df_us_holidays["Date"].dt.strftime('%Y-%m-%d')): 
        return row["PREDICTED_REVENUE"] * 0.8
    elif row["DATE"].weekday() == 5 or row["DATE"].weekday() == 6: #Saturday/Sundays
        return row["PREDICTED_REVENUE"] * 1.2
    return row["PREDICTED_REVENUE"]

In [28]:
df_forecast_pandas = df_forecast.move_to("pandas")

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
# Adjust for holidays
df_forecast_pandas["PREDICTED_REVENUE"] = df_forecast_pandas.apply(adjust_for_holiday_weekend, axis=1)

In [30]:
df_forecast["PREDICTED_REVENUE"] = df_forecast.apply(adjust_for_holiday_weekend, axis=1)

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
type(df_forecast)

modin.pandas.dataframe.DataFrame

In [32]:
import altair as alt
alt.data_transformers.disable_max_rows()
df_forecast = df_forecast.move_to("pandas")
chart_predicted = alt.Chart(df_forecast).mark_line(color='blue').encode(
    x='monthdate(DATE):T',
    y='PREDICTED_REVENUE:Q',
    tooltip=['DATE', 'PREDICTED_REVENUE']
).properties(title="Predicted Revenue")
chart_predicted

alt.Chart(...)

In [33]:
df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]
df_transactions_filtered_groupby = df_transactions_filtered.groupby("DATE")["REVENUE"].mean()
# df_transactions_filtered_groupby = df_transactions_filtered_groupby.move_to("pandas")
chart_last_year = alt.Chart(df_transactions_filtered_groupby).mark_line(color='red').encode(
    x='monthdate(DATE):T',
    y='REVENUE:Q',
    tooltip=['DATE', 'REVENUE']
).properties(title="Last Year Revenue")

# Overlay the charts
final_chart = chart_predicted + chart_last_year
final_chart

ValueError: DATE encoding field is specified without a type; the type cannot be automatically inferred because the data is not specified as a pandas.DataFrame.

alt.LayerChart(...)